In [1]:
import torch
from transformers import AutoModel, AutoTokenizer


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/koen/miniconda3/envs/toolbox/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/koen/miniconda3/envs/toolbox/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/koen/miniconda3/envs/toolbox/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    s

In [2]:

tokenizer = AutoTokenizer.from_pretrained(
    "nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True
)
model = AutoModel.from_pretrained(
    "nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

<All keys matched successfully>


NomicBertModel(
  (embeddings): NomicBertEmbeddings(
    (word_embeddings): Embedding(30528, 768)
    (token_type_embeddings): Embedding(2, 768)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (emb_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (encoder): NomicBertEncoder(
    (layers): ModuleList(
      (0-11): 12 x NomicBertBlock(
        (attn): NomicBertAttention(
          (rotary_emb): NomicBertRotaryEmbedding()
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=False)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (mlp): NomciBertGatedMLP(
          (fc11): Linear(in_features=768, out_features=3072, bias=False)
          (fc12): Linear(in_features=768, out_features=3072, bias=False)
          (fc2): Linear(in_features=3072, out_features=768, bias=False)
          (norm): Identity()
        )
        (dropout1): Dropout(p=0.0, inplace=False)
        (norm1

In [3]:
inputs = tokenizer(
        ["ABC", "DEF"],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,
    )
inputs = {k: v.to(device) for k, v in inputs.items()}


with torch.no_grad():
    outputs = model(**inputs)
    # Use the mean pooling of the last hidden state as embedding
    last_hidden = outputs.last_hidden_state
    attention_mask = inputs["attention_mask"]
    mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
    sum_embeddings = torch.sum(last_hidden * mask_expanded, 1)
    sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
    embeddings = (sum_embeddings / sum_mask).squeeze().tolist()

In [8]:
[(x,e) for x, e in zip(["a", "b"], embeddings)]

[('a',
  [-0.2812548875808716,
   0.6373253464698792,
   -2.823735475540161,
   -1.7042937278747559,
   -0.3461134433746338,
   0.5669879913330078,
   -0.7702401280403137,
   -0.4867357313632965,
   1.8637996912002563,
   -1.3147755861282349,
   -0.344700425863266,
   1.246450424194336,
   1.8009778261184692,
   1.275769591331482,
   1.021206259727478,
   -1.4459902048110962,
   -0.5658994317054749,
   -0.24589093029499054,
   0.1970137357711792,
   -0.7519738078117371,
   0.1706324815750122,
   -1.3815535306930542,
   -0.18297477066516876,
   0.16262473165988922,
   2.9186229705810547,
   0.802720308303833,
   -0.6538533568382263,
   0.3850266635417938,
   -1.196873664855957,
   1.5393625497817993,
   -0.41676077246665955,
   -1.2388700246810913,
   -0.2070664018392563,
   -2.268739938735962,
   -0.17440007627010345,
   -0.8431535363197327,
   1.308164119720459,
   1.3214572668075562,
   -1.4764143228530884,
   -0.20540332794189453,
   1.0208375453948975,
   1.1599751710891724,
   -0.